# Importando as Bibliotecas

In [1]:
import ranking

import pandas as pd
import numpy as np
import string

from scipy import spatial

from sklearn.feature_extraction.text import CountVectorizer

# Lendo os Datasets

In [2]:
arquivos = ['celulares']
#arquivos = ['fogoes']
#arquivos = ['geladeiras']
#arquivos = ['notebooks']
#arquivos = ['tvs']

lista_df_pp = []
for arquivo in arquivos:

    df_pp = pd.read_csv(f"Dados/Produtos Processados/pp_{arquivo}.csv", dtype = {'ean': str})
    lista_df_pp.append(df_pp)

In [3]:
df_pp = lista_df_pp[0]
df_pp.head(1)

,categoria,descricao,ean,preco,titulo,url,titulo_cb,loja
0,/categoria/celulares-e-smartphones/smartphone/...,Smartphone Samsung Galaxy S22 Ultra 512GB 5G c...,7892509122580,"R$ 8.858,63",Smartphone Samsung Galaxy S22 Ultra 512GB 5G c...,https://www.americanas.com.br/produto/4987525162,smartphone samsung galaxy s22 ultra 512gb 5g c...,americanas


# Aplicando o BoW

In [4]:
# removendo a pontuação do título
df_pp["titulo_pp"] = df_pp["titulo"].apply(lambda x: ranking.remove_pontuacao(x))

# calculando o tamanho máximo do título
tam_max = max(df_pp.apply(lambda row: len(row["titulo_pp"]), axis = 1))

In [5]:
# calculando o BoW do título
cv, titulo_bow = ranking.formatar_entrada_bow(df_pp['titulo_pp'], mf = tam_max)
#np.unique(titulo_bow)

Calculando a distância entre os vetores

In [6]:
resultado = ranking.calcular_dis_2_vetores(titulo_bow)

c:\Users\llvs2\anaconda3\lib\site-packages\scipy\spatial\distance.py:699: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Colocando o resultado em ordem (menor distância até maior distância)

In [8]:
indices, valores = ranking.ordenar_resultado(resultado)

Calculando o acccuracy@k

In [9]:
for k in [1, 10, 50]:

    # criando uma coluna nova no df
    df_pp[f'k-{k}'] = 0
    df_pp[f'k-{k}-qtd'] = 0
    df_pp[f'k-{k}-qtd/{k}'] = 0

    for i in range(len(indices)):

        for j in range(k):
            
            if df_pp['ean'].loc[i] == df_pp['ean'].loc[indices[i][j]]:

                df_pp[f'k-{k}'].loc[i] = 1
                df_pp[f'k-{k}-qtd'].loc[i] += 1

        df_pp[f'k-{k}-qtd/{k}'].loc[i] = (df_pp[f'k-{k}-qtd'].loc[i])/k
        

c:\Users\llvs2\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [ ]:
df_pp['k-1'].value_counts()

In [ ]:
df_pp['k-10'].value_counts()

In [ ]:
df_pp['k-50'].value_counts()

In [ ]:
df_pp[["k-1", "k-1-qtd", "k-1-qtd/1", "k-10", "k-10-qtd", "k-10-qtd/10", "k-50", "k-50-qtd", "k-50-qtd/50"]]

In [10]:
df_pp[["k-1", "k-1-qtd", "k-1-qtd/1", "k-10", "k-10-qtd", "k-10-qtd/10", "k-50", "k-50-qtd", "k-50-qtd/50"]].describe()

,k-1,k-1-qtd,k-1-qtd/1,k-10,k-10-qtd,k-10-qtd/10,k-50,k-50-qtd,k-50-qtd/50
count,2079.000000,2079.000000,2079.000000,2079.000000,2079.00000,2079.000000,2079.000000,2079.000000,2079.000000
mean,0.003367,0.003367,0.003367,0.097643,0.12025,0.012025,0.152477,0.210197,0.004204
std,0.057942,0.057942,0.057942,0.296903,0.40327,0.040327,0.359569,0.570001,0.011400
min,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,4.00000,0.400000,1.000000,4.000000,0.080000


In [11]:
# criando uma coluna nova no df
df_pp['match_rank'] = 0
df_pp['1/match_rank'] = 0
df_pp['match_rank/total'] = 0
tam_df = df_pp.shape[0]

# para cada linha do dataframe
for i in range(tam_df):

    # para cada uma das distâncias encontradas rankeadas em ordem decrescente
    for cont, j in enumerate(indices[i]):
    
        # se for match
        if (df_pp['ean'].loc[i] == df_pp['ean'].loc[j]) and (i != j):

            # guarda a posição do primeiro match, lembrando que o enumerate começa em 0
            df_pp['match_rank'].loc[i] = (cont + 1)
            df_pp['1/match_rank'].loc[i] = 1/(cont + 1)
            df_pp['match_rank/total'].loc[i] = (cont + 1)/tam_df
            
            # para o for
            break

c:\Users\llvs2\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [12]:
df_pp[["k-1", "k-1-qtd", "k-1-qtd/1", "k-10", "k-10-qtd", "k-10-qtd/10", "k-50", "k-50-qtd", "k-50-qtd/50", 'match_rank', 'match_rank/total', '1/match_rank']]

,k-1,k-1-qtd,k-1-qtd/1,k-10,k-10-qtd,k-10-qtd/10,k-50,k-50-qtd,k-50-qtd/50,match_rank,match_rank/total,1/match_rank
0,0,0,0,1,1,0.1,1,1,0.02,9,0.004329,0.111111
1,0,0,0,0,0,0.0,0,0,0.00,0,0.000000,0.000000
2,0,0,0,0,0,0.0,0,0,0.00,0,0.000000,0.000000
3,0,0,0,1,3,0.3,1,3,0.06,7,0.003367,0.142857
4,0,0,0,0,0,0.0,0,0,0.00,0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2074,0,0,0,0,0,0.0,0,0,0.00,0,0.000000,0.000000
2075,0,0,0,0,0,0.0,0,0,0.00,70,0.033670,0.014286
2076,0,0,0,0,0,0.0,0,0,0.00,0,0.000000,0.000000
2077,0,0,0,0,0,0.0,0,0,0.00,0,0.000000,0.000000


In [ ]:
#tirando a média e dividindo pela quantidade total de produtos
#df_pp[['match_rank', 'match_rank/total', '1/match_rank']].describe()

In [ ]:
#df_pp[["k-1", "k-10", "k-50", "match_rank"]][df_pp['match_rank'] > 50]

In [ ]:
#df_pp.to_csv(f"Dados/Produtos Processados/Metricas/{arquivos[0]}.csv", index = False)